In [ ]:
from flask import Flask,render_template,request,redirect,url_for,session
from flask_pymongo import PyMongo
from werkzeug.utils import secure_filename
from web3 import Web3, HTTPProvider
from bson.objectid import ObjectId#to retrieve mongodb objectid
from werkzeug.utils import secure_filename

import json
import os
import ast
from flask import flash

# from decorators import login_required
from functools import wraps




from vyper import compile_codes
contract_source_code = '''
value: public(wei_value) #Value of the item
seller: public(address)
buyer: public(address)
unlocked: public(bool)
ended: public(bool)

@public
@payable
def __init__():
    assert (msg.value % 2) == 0
    self.value = msg.value / 2  # The seller initializes the contract by
        # posting a safety deposit of 2*value of the item up for sale.
    self.seller = msg.sender
    self.unlocked = True

@public
def abort():
    assert self.unlocked #Is the contract still refundable?
    assert msg.sender == self.seller # Only the seller can refund
        # his deposit before any buyer purchases the item.
    selfdestruct(self.seller) # Refunds the seller and deletes the contract.

@public
@payable
def purchase():
    assert self.unlocked # Is the contract still open (is the item still up
                         # for sale)?
    assert msg.value == (2 * self.value) # Is the deposit the correct value?
    self.buyer = msg.sender
    self.unlocked = False

@public
def received():
    # 1. Conditions
    assert not self.unlocked # Is the item already purchased and pending
                             # confirmation from the buyer?
    assert msg.sender == self.buyer
    assert not self.ended

    # 2. Effects
    self.ended = True

    # 3. Interaction
    send(self.buyer, self.value) # Return the buyer's deposit (=value) to the buyer.
    selfdestruct(self.seller) # Return the seller's deposit (=2*value) and the
                              # purchase price (=value) to the seller.
'''
smart_contract = {}
smart_contract['test'] = contract_source_code
#在這邊將所有的contract 取一樣的名字test(just an object name)  但是當被deploy時會有不同的contractAddress 並寫入資料庫
format = ['abi', 'bytecode']
compiled_code = compile_codes(smart_contract, format, 'dict')

bytecode = compiled_code['test']['bytecode']
abi = compiled_code['test']['abi']


w3 = Web3(HTTPProvider('http://localhost:7545'))




app = Flask(__name__)
app.config['SECRET_KEY'] = os.urandom(24)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)

def login_required(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print('session get account: ',session.get('account'))
        if session.get('account'):
            return func(*args,**kwargs)
        else:
            return redirect(url_for('login'))
        
    return wrapper

@app.route('/')
def index():
    hotel_collection=mongo.db.hotel_id
    context = {#find()return list item
        'hotels':[i for i in hotel_collection.find({}).limit(10)]\
    }
    return render_template('index.html',**context) 
@app.route('/next/<next_id>')
def nextpage(next_id):
    hotel_collection=mongo.db.hotel_id
    context = {#find()return list item
        'hotels':[i for i in hotel_collection.find({'hotelid':{'$gt':int(next_id)}}).limit(10)]\
    }
    
    return render_template('index.html',**context) 
@app.route('/pre/<pre_id>')
def prepage(pre_id):
    hotel_collection=mongo.db.hotel_id
    if int(pre_id)<=10:
        context = {#find()return list item
            'hotels':[i for i in hotel_collection.find({}).limit(10)]\
        }
    else:
        context = {#find()return list item
            'hotels':[i for i in hotel_collection.find({'hotelid':{'$gte':int(pre_id)-10}}).limit(10)]\
        }
    
    return render_template('index.html',**context) 


#     hotel_name=[]
#     hotel_url=[]
#     hotel_id = []
#     j=0
#     with open('city_hotelname_hid_hurl','r',encoding='utf-8') as f:
#         for line in f:
#             a = line.split()
            
#             hotel_name.append(''.join(a[1:len(a)-2]))
#             hotel_url.append(a[len(a)-1])
#             hotel_id.append(a[len(a)-2])
#             if j==10:
#                 break
#             else:
#                 j+=1
#     return render_template('index.html',hotel_name=hotel_name,hotel_url=hotel_url)

@app.route('/login/',methods=['GET','POST'])
def login():
    if request.method == 'GET':
        return render_template('login.html')
    else:
        account = request.form.get('account')
        password = request.form.get('password')
        user_collection=mongo.db.user
        user = user_collection.find_one({"$and":[ {'account':account}
                                             ,{'password':password}
                                             ]})#查無回傳None
        if user:
            session['account'] = user['account']
            session.permanent = True #如果想在31天都不需要登入
            
            return redirect(url_for('index'))
        else:
            return '名稱或密碼錯誤'

@app.route("/logout/", methods=['GET','POST'])
def logout():
    session['account'] = None
    return redirect('/')
@app.route('/register/',methods=['GET','POST'])
def register():
    if request.method == 'GET':
        return render_template('register.html')
    
    
    else:
        account = request.form.get('account')
        password1 = request.form.get('password1')
        password2 = request.form.get('password2')
        
        user_collection=mongo.db.user
        uid = user_collection.estimated_document_count()+1
        if user_collection.find_one({'account':account}):#查無回傳None
            return '該名稱已被註冊，請更換'
        else:
            if password1!=password2:
                return '兩次密碼不相同，請核對後再填寫'
            else:
                user_collection.insert_one({'account':account,\
                            'uid':uid,\
                            'password':password1,\
                            'public_key':'',\
                            'private_key':'',\
                            'ratings':[],\
                            'rcmhotel':[],\
                            'eth':100
                        })
                return redirect(url_for('login'))
        
# @app.route('/login/<id>')
# def login0(id):
#     return render_template('login.html',id=id)

@app.route('/addhotel/',methods=['GET','POST'])
@login_required
def addhotel():
    if request.method =='GET':
        return render_template('addhotel.html')
    else:
        hotelname = request.form.get('hotelname')
        address = request.form.get('address')
        introduction = request.form.get('introduction')
        price = request.form.get('price')
        f = request.files['file']
        basepath = os.getcwd()  # 當前文件所再路徑
        upload_path = os.path.join(basepath, 'static/images/hotelimg/',secure_filename(session.get('account')+f.filename))
        if f.filename:
            imgpath = 'images/hotelimg/'+secure_filename(session.get('account')+f.filename)
        else:
            imgpath = ''
            
#         upload_path = os.path.join(basepath, 'static\images',secure_filename(file.filename))  
#         #注意：沒有的文件一定要先創見 不然會提示沒有該文件
        f.save(upload_path)
        landlord = session.get('account')#房東
        user_collection = mongo.db.user
        publickey = user_collection.find_one({'account':session.get('account')}).get('public_key')
        
#         建立合約 取得contractAddress*********************
        #w3 = Web3(HTTPProvider('http://localhost:7545'))
        Test = w3.eth.contract(abi=abi, bytecode=bytecode)#Test -->smart contract object name
        tx_hash = Test.constructor().transact({'from': publickey,'value': w3.toWei(str(int(price)*2), 'ether')})#乘2  包含訂金（設跟price相同）
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)#建立合約後等待收據
        print(tx_receipt)
        contractAddress = tx_receipt["contractAddress"]#here we get the contract address
        print(contractAddress)
        
        
        Test = w3.eth.contract(address=contractAddress , abi=abi)
        print(w3.fromWei(Test.functions.value().call(),'ether'))
        print(Test.functions.seller().call())
        print(Test.functions.buyer().call())
        print(Test.functions.unlocked().call())
        
        rent_hotel_collection=mongo.db.rent_hotel
        rent_hotel_collection.insert_one({'landlord':landlord,\
                            'publickey':publickey,\
                            'hotelname':hotelname,\
                            'address':address,\
                            'introduction':introduction,\
                            'price':price,\
                            'ratings':[],\
                            'imgpath':imgpath,
                            'contractAddress':contractAddress.lower(),#轉小寫存入
                            'status':'sale'  #we need to record the state of the hotel  (slae  ,   purchased ,  confirmed)
                        })
        transaction_collection=mongo.db.transaction
        transaction_collection.insert_one({'blockNumber':tx_receipt.get('blockNumber'),\
                'transactionIndex':tx_receipt.get('transactionIndex'),\
                'hotelname': rent_hotel_collection.find_one({"contractAddress" : contractAddress.lower()}).get('hotelname'),#contractAddress.lower()
                'from':tx_receipt.get('from'),\
                'to':tx_receipt.get('to'),\
                'cost':'sale'
            })

        
        
        
        return redirect(url_for('renthotel'))
    

##################
#################
@app.route('/profile/')
@login_required
def profile():
    renthotel_collection=mongo.db.rent_hotel
    user_collection=mongo.db.user
    transaction_collection=mongo.db.transaction
    publickey = user_collection.find_one({'account':session.get('account')}).get('public_key')
    #w3 = Web3(HTTPProvider('http://localhost:7545'))
    find_transaction = [i for i in transaction_collection.find({'from':publickey.lower()}).sort('_id',-1)]#lower轉小寫字母
    booked = []
    
    for i in find_transaction:
        print(i.get('to'))#contractAddress
        if i.get('cost')=='abort' or i.get('cost')=='sale':#將abort txn and sale(addhotel)過濾
            continue
        booked.append(renthotel_collection.find_one({'contractAddress':i.get('to')}))
    print('booked',booked)
    
    context = {
        'booked' :[i for i in booked],
        'rent_hotels':[i for i in renthotel_collection.find({'landlord':session.get('account')}).sort('_id',-1)],\
        'publickey':publickey,\
        'privatekey':user_collection.find_one({'account':session.get('account')}).get('private_key'),\
        'balance':w3.fromWei(w3.eth.getBalance(publickey), 'ether'),\
        'transactions':[i for i in transaction_collection.find({'$or':[{'from':publickey.lower()},{'to':publickey.lower()}]}).sort('_id',-1)]#lower轉小寫字母
    }
    return render_template('profile.html',**context)



@app.route('/renthotel/')
def renthotel():
    rent_hotel_collection=mongo.db.rent_hotel
    context = {#find()return list item
        'rent_hotels':[i for i in rent_hotel_collection.find({'status':'sale'}).sort('_id',-1)]\
    }
    return render_template('newhotel.html',**context)
###############
###############
@app.route('/sendmoney/<renthotel_id>/',methods=['GET','POST'])
@login_required
def sendmoney(renthotel_id):
    rent_hotel_collection=mongo.db.rent_hotel
    user_collection = mongo.db.user
    rent_hotel = rent_hotel_collection.find_one({"_id" : ObjectId(renthotel_id)})
    hotelname = rent_hotel.get('hotelname')
    landlord =  rent_hotel.get('landlord')
    price =  rent_hotel.get('price')
    to =  rent_hotel.get('publickey')
    contractAddress = rent_hotel.get('contractAddress')
    status = rent_hotel.get('status')
    if request.method =='GET':
        
        context = {
            'hotelname':hotelname,\
            'landlord':landlord,\
            'price':price,\
            'from':user_collection.find_one({'account':session.get('account')}).get('public_key'),\
            'to':to,
            'contractAddress':contractAddress
        }
        return render_template('sendmoney.html',**context)
    elif status!="sale":
        rent_hotel_collection=mongo.db.rent_hotel
        context = {#find()return list item
            'rent_hotels':[i for i in rent_hotel_collection.find({}).sort('_id',-1)]\
        }
        return render_template('newhotel.html',**context)
    else:
        frompublickey = user_collection.find_one({'account':session.get('account')}).get('public_key')
        fromprivatekey = user_collection.find_one({'account':session.get('account')}).get('private_key')
        topublickey = to
        vcoin = price
        
        #w3 = Web3(HTTPProvider('http://localhost:7545'))

        private_key = fromprivatekey
        nonce = w3.eth.getTransactionCount(frompublickey)
        
        
        #purchase
        address =w3.toChecksumAddress(contractAddress) 
# Web3 accept only checksum address. So we should ensure the given address is a
# checksum address before accessing the corresponding contract.
        Test = w3.eth.contract(address=address , abi=abi)
        print(w3.fromWei(Test.functions.value().call(),'ether'))
        print(Test.functions.seller().call())
        print(Test.functions.buyer().call())
        print(Test.functions.unlocked().call())
        print(landlord,address)
        
        txn = Test.functions.purchase().buildTransaction({#transaction
            'value': w3.toWei(int(vcoin)*2, 'ether'),#vocin 包含訂金==price
            'gas': 100000,
            'gasPrice': w3.toWei('1', 'gwei'),
            'nonce': nonce
              })

        signed_txn = w3.eth.account.signTransaction(txn, private_key=private_key)
        signed_txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        txn_rec = w3.eth.waitForTransactionReceipt(signed_txn_hash)

        print(w3.fromWei(Test.functions.value().call(),'ether'))
        print(Test.functions.seller().call())
        print(Test.functions.buyer().call())
        print(Test.functions.unlocked().call())
#blow only send money did not use smart contract
#         transaction = {
#           'to': Web3.toChecksumAddress(topublickey),
#           'value': w3.toWei(vcoin, 'ether'),
#           'gas': 100000,
#           'gasPrice': w3.toWei(vcoin, 'gwei'),
#           'nonce': nonce
#         }

#         signed = w3.eth.account.signTransaction(transaction, private_key)
#         tx = w3.eth.sendRawTransaction(signed.rawTransaction)
#         transaction = w3.eth.getTransactionReceipt(tx)
            
        context = {
            'blockNumber':txn_rec.get('blockNumber'),\
            'transactionIndex':txn_rec.get('transactionIndex'),\
            'hotelname': rent_hotel_collection.find_one({"_id" : ObjectId(renthotel_id)}).get('hotelname'),\
            'from':txn_rec.get('from'),\
            'to':txn_rec.get('to') ,\
            'cost':vcoin,\
            'balance':str(w3.fromWei(w3.eth.getBalance(frompublickey ), 'ether'))
        }
#      'to':transaction.get('to'),\
        transaction_collection=mongo.db.transaction
        transaction_collection.insert_one({'blockNumber':txn_rec.get('blockNumber'),\
            'transactionIndex':txn_rec.get('transactionIndex'),\
            'hotelname': rent_hotel_collection.find_one({"_id" : ObjectId(renthotel_id)}).get('hotelname'),\
            'from':txn_rec.get('from'),\
            'to':txn_rec.get('to'),\
            'cost':vcoin
            })
        renthotel_collection = mongo.db.rent_hotel
        renthotel_collection.update_one({"_id" : ObjectId(renthotel_id)},{'$set':{'status':'purchase'}})
        context = json.dumps(context)
        return redirect(url_for('getReceipt',context=context))
        
@app.route('/transactionReceipt/<context>')  
@login_required
def getReceipt(context):
#     print(context)
    context = json.loads(context) 
    return  render_template('transactionReceipt.html',**context)
##########################
##########################
@app.route('/recommend')
@login_required
def recommend():
    account = session.get('account')
    rcmhotel_collection=mongo.db.user
    rcmhotel = rcmhotel_collection.find_one({'account':session.get('account')})
    
    hotel_collection=mongo.db.hotel_id
    hotelid = [list(i.keys())[0] for i in rcmhotel['rcmhotel']]#list
    hotels = []
    for i in hotelid:
        hotels.append(hotel_collection.find_one({'hotelid': int(i) }))
    context = {
        'hotels':[i for i in hotels]\
    }
    print(context)
#     context = {
#         'hotels':[i for i in hotel_collection.find({'hotelid':{'$in':hotelid}})]\
#     }
    return render_template('index.html',**context) 
    
#     for i in rcmhotel['rcmhotel']:
#         hotel_name.append(list(i.values())[0])
#     return render_template('index.html',hotel_name=hotel_name,hotel_url=hotel_url)


@app.route('/deletehotel/')
def deletehotel():
    rent_hotel_collection=mongo.db.rent_hotel
    contractAddress = request.args.get('contractAddress')
    aborthotel = rent_hotel_collection.find_one({'$and':[{'contractAddress':contractAddress},{'status':'sale'}]})
    if aborthotel:#確認有無可以刪除的旅館
        address =Web3.toChecksumAddress(contractAddress)
        Test = w3.eth.contract(address=address , abi=abi)
        print(w3.fromWei(Test.functions.value().call(),'ether'))
        print(Test.functions.seller().call())
        print(Test.functions.buyer().call())
        print(Test.functions.unlocked().call())
        print(w3.eth.defaultAccount)

        frompublickey = user_collection.find_one({'account':session.get('account')}).get('public_key')
        fromprivatekey = user_collection.find_one({'account':session.get('account')}).get('private_key')
        private_key = fromprivatekey
        nonce = w3.eth.getTransactionCount(frompublickey)

        txn = Test.functions.abort().buildTransaction({
            'gas': 100000,
            'gasPrice': w3.toWei('1', 'gwei'),
            'nonce': nonce
              })
        signed_txn = w3.eth.account.signTransaction(txn, private_key=private_key)
        signed_txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        txn_rec = w3.eth.waitForTransactionReceipt(signed_txn_hash)
        print(txn_rec)
        transaction_collection=mongo.db.transaction
        transaction_collection.insert_one({'blockNumber':txn_rec.get('blockNumber'),\
                'transactionIndex':txn_rec.get('transactionIndex'),\
                'hotelname': rent_hotel_collection.find_one({"contractAddress" : contractAddress}).get('hotelname'),\
                'from':txn_rec.get('from'),\
                'to':txn_rec.get('to'),\
                'cost':'abort'
            })
    
    
        rent_hotel_collection=mongo.db.rent_hotel
        hotel_id = ObjectId(request.args.get('hotel_id'))
        account = session.get('account')
        deletehotel = rent_hotel_collection.find_one({'$and':[{'landlord':account},{'_id':hotel_id}]})
        if deletehotel:
            rent_hotel_collection.remove(deletehotel)
        return redirect(url_for('profile'))
    else:
        return redirect(url_for('profile'))
    
@app.route('/received/')
def received():
    rent_hotel_collection=mongo.db.rent_hotel
    contractAddress = request.args.get('contractAddress')
    address =w3.toChecksumAddress(contractAddress) 
    Test = w3.eth.contract(address=address , abi=abi)
    print(w3.fromWei(Test.functions.value().call(),'ether'))
    print(Test.functions.seller().call())
    print(Test.functions.buyer().call())
    print(Test.functions.unlocked().call())
    print(w3.eth.defaultAccount)
    
    frompublickey = user_collection.find_one({'account':session.get('account')}).get('public_key')
    fromprivatekey = user_collection.find_one({'account':session.get('account')}).get('private_key')
    private_key = fromprivatekey
    nonce = w3.eth.getTransactionCount(frompublickey)
    
    txn = Test.functions.received().buildTransaction({
        'gas': 100000,
        'gasPrice': w3.toWei('1', 'gwei'),
        'nonce': nonce
          })
    signed_txn = w3.eth.account.signTransaction(txn, private_key=private_key)
    signed_txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    txn_rec = w3.eth.waitForTransactionReceipt(signed_txn_hash)
    print(txn_rec)
    transaction_collection=mongo.db.transaction
    transaction_collection.insert_one({'blockNumber':txn_rec.get('blockNumber'),\
            'transactionIndex':txn_rec.get('transactionIndex'),\
            'hotelname': rent_hotel_collection.find_one({"contractAddress" : contractAddress}).get('hotelname'),\
            'from':txn_rec.get('from'),\
            'to':txn_rec.get('to'),\
            'cost':'received'
        })
    
   
    renthotel_collection.update_one({"contractAddress" : contractAddress},{'$set':{'status':'comfirmed'}})
    return redirect(url_for('profile'))

if __name__=='__main__':    
    app.run(host="0.0.0.0", port=8090,debug=True,use_reloader=False) 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:8090/ (Press CTRL+C to quit)
120.110.113.93 - - [04/Dec/2019 15:45:02] "GET /received/?contractAddress=0x84099e79ef2569f7b7fc82e88bccc9c2d6bb16bf HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/home/hduser/anaconda3/envs/flask_env/lib/python3.7/site-packages/web3/contract.py", line 1357, in call_contract_function
    output_data = web3.codec.decode_abi(output_types, return_data)
  File "/home/hduser/anaconda3/envs/flask_env/lib/python3.7/site-packages/eth_abi/codec.py", line 179, in decode_abi
    return decoder(stream)
  File "/home/hduser/anaconda3/envs/flask_env/lib/python3.7/site-packages/eth_abi/decoding.py", line 127, in __call__
    return self.decode(stream)
  File "/home/hduser/anaconda3/envs/flask_env/lib/python3.7/site-packages/eth_utils/functional.py", line 45, in inner
    return callback(fn(*args, **kwargs))
  File "/home/hduser/anaconda3/envs/flask_env/lib/python3.7/site-packages/eth_abi/decoding.py", line 173, in decode
    yield

session get account:  None


120.110.113.93 - - [04/Dec/2019 15:45:10] "POST /login/ HTTP/1.1" 302 -
120.110.113.93 - - [04/Dec/2019 15:45:10] "GET / HTTP/1.1" 200 -
120.110.113.93 - - [04/Dec/2019 15:45:12] "GET /addhotel/ HTTP/1.1" 200 -


session get account:  user1
session get account:  user1
AttributeDict({'transactionHash': HexBytes('0x3fa003acc382a82e21e6ac4304d37d60b35d2d2408fb14e11609521c3be686e1'), 'transactionIndex': 0, 'blockHash': HexBytes('0x3bf79fa43e4c8852171c70d5222ee9ecb9318b02023831b795a96a7a2f958d7e'), 'blockNumber': 69, 'from': '0x6db87088ba7d069ba8fba685f50705865176ca3e', 'to': None, 'gasUsed': 281356, 'cumulativeGasUsed': 281356, 'contractAddress': '0xdd0adCca6c9cc344b57E776CaB0e0206c9D7FD11', 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

...

0x0000000000000000000000000000000000000000
True


36.232.52.173 - - [05/Dec/2019 05:15:03] "GET /renthotel/ HTTP/1.1" 200 -
36.232.52.173 - - [05/Dec/2019 05:15:09] "GET /logout/ HTTP/1.1" 302 -
36.232.52.173 - - [05/Dec/2019 05:15:09] "GET / HTTP/1.1" 200 -
36.232.52.173 - - [05/Dec/2019 05:15:10] "GET /login/ HTTP/1.1" 200 -
36.232.52.173 - - [05/Dec/2019 05:15:15] "POST /login/ HTTP/1.1" 302 -
36.232.52.173 - - [05/Dec/2019 05:15:15] "GET / HTTP/1.1" 200 -
36.232.52.173 - - [05/Dec/2019 05:15:19] "GET /profile/ HTTP/1.1" 200 -


session get account:  user1
None
0x8ed5ad2688d26596e59dacfa553dd529c2295b97
None
0x9fa51782320c8b7dbb43616fb39f137cde1aa573
0x9fa51782320c8b7dbb43616fb39f137cde1aa573
None
booked [{'_id': ObjectId('5de7ebba10b708ef96ed0874'), 'landlord': 'user1', 'publickey': '0x6DB87088BA7D069BA8Fba685f50705865176Ca3E', 'hotelname': 'test hotel', 'address': 'test address', 'introduction': 'test', 'price': '2', 'ratings': [], 'imgpath': '', 'contractAddress': '0x8ed5ad2688d26596e59dacfa553dd529c2295b97', 'status': 'purchase'}, {'_id': ObjectId('5de7ba4010b708ef96ed0870'), 'landlord': 'user1', 'publickey': '0x6DB87088BA7D069BA8Fba685f50705865176Ca3E', 'hotelname': 'w', 'address': 'w', 'introduction': 'w price 2 deposite same as price 2', 'price': '2', 'ratings': [], 'imgpath': '', 'contractAddress': '0x9fa51782320c8b7dbb43616fb39f137cde1aa573', 'status': 'comfirmed'}, {'_id': ObjectId('5de7ba4010b708ef96ed0870'), 'landlord': 'user1', 'publickey': '0x6DB87088BA7D069BA8Fba685f50705865176Ca3E', 'hotelname'

36.232.52.173 - - [05/Dec/2019 05:16:29] "GET /received/?contractAddress=0x8ed5ad2688d26596e59dacfa553dd529c2295b97 HTTP/1.1" 302 -
36.232.52.173 - - [05/Dec/2019 05:16:29] "GET /profile/ HTTP/1.1" 200 -


AttributeDict({'transactionHash': HexBytes('0x12d1288cd8e05fff6255d3b4a2eedc06c467d99a7ae49e3b742b59cd3112da72'), 'transactionIndex': 0, 'blockHash': HexBytes('0x8b92a46bb4e6ff0555e6058591e5d7c01205fdb1281433c6cd4bad3d3bdcc10c'), 'blockNumber': 7, 'from': '0x6db87088ba7d069ba8fba685f50705865176ca3e', 'to': '0x8ed5ad2688d26596e59dacfa553dd529c2295b97', 'gasUsed': 31183, 'cumulativeGasUsed': 31183, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'v': '0x0

In [1]:
from flask import Flask
from flask_pymongo import PyMongo

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)
hotelname = []
hotelurl = []
rcmhotel_collection=mongo.db.user
rcmhotel = rcmhotel_collection.find_one({'account':'charooooonc'})
 
for i in rcmhotel['rcmhotel']:
    hotelname.append(list(i.values())[0])
    print(list(i.values())[0])

print(hotelname)


異宿風巢景觀旅店 台灣安平
長成民宿 台灣澎湖
花蓮莊家堡田園民宿 台灣吉安
綠原度假旅社 台灣雲林
逸心園民宿 台灣壽豐
旅人船說背包聚落 台灣中西區
旅捷商務旅館 台灣苓雅
Z-house市政府捷運站套房 台灣信義區
雅思德商務公寓 台灣中和
奇豆民宿 台灣花蓮市
['異宿風巢景觀旅店 台灣安平', '長成民宿 台灣澎湖', '花蓮莊家堡田園民宿 台灣吉安', '綠原度假旅社 台灣雲林', '逸心園民宿 台灣壽豐', '旅人船說背包聚落 台灣中西區', '旅捷商務旅館 台灣苓雅', 'Z-house市政府捷運站套房 台灣信義區', '雅思德商務公寓 台灣中和', '奇豆民宿 台灣花蓮市']


In [4]:
from flask import Flask
from flask_pymongo import PyMongo

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)
user_collection=mongo.db.rent_hotel
context = {
        'rent_hotels':[i for i in user_collection.find({'landlord':'user1'}).sort('_id',-1)]
}
# for i in context.items():
#     print(i)
print(context['rent_hotels'][0])
print(context['rent_hotels'][0]['introduction'])

[]


TypeError: list indices must be integers or slices, not str

In [70]:
from flask import Flask
from flask_pymongo import PyMongo

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/recommend"
mongo = PyMongo(app)
hotel_id_collection = mongo.db.hotel_id
hotel_id = hotel_id_collection.find({})
for i in hotel_id:
    print(i.get('hotelname'),\
          i.get('landlord'),\
          i.get('location'))#get取代['key']回傳none
    

新驛旅店-復興北路店 None 台灣中山區
優美飯店 None 台灣中山區
圓山大飯店 None 台灣中山區
老爺會館台北林森 None 台灣中山區
華國大飯店 None 台灣中山區
丰居旅店雙連館 None 台灣中山區
方舟旅店-長安復興 None 台灣中山區
台北澄舍商旅 None 台灣中山區
喜瑞飯店 None 台灣中山區
雀客旅館 None 台灣中山區


In [18]:
os.getcwd()

'/home/hduser/下載/recommend_system-master/final_recmhotel'

In [10]:
help(os.path.dirname)

Help on function dirname in module posixpath:

dirname(p)
    Returns the directory component of a pathname



In [14]:
hotel_collection=mongo.db.hotel_id
    context = {#find()return list item
        'hotels':[i for i in hotel_collection.find({'_id':{'$ls':ObjectId(pre_id)}}).limit(10)]\
    }

In [23]:
format(12,'x')

'c'

In [36]:
rcmhotel_collection=mongo.db.user
rcmhotel = rcmhotel_collection.find_one({'account':'charooooonc'})
hotel_name = [] 
for i in rcmhotel['rcmhotel']:
    hotel_name.append(list(i.values())[0])
for i in rcmhotel.get('rcmhotel'):
    print(i)

{'1984': '異宿風巢景觀旅店 台灣安平'}
{'5914': '長成民宿 台灣澎湖'}
{'5233': '花蓮莊家堡田園民宿 台灣吉安'}
{'3394': '綠原度假旅社 台灣雲林'}
{'5042': '逸心園民宿 台灣壽豐'}
{'1800': '旅人船說背包聚落 台灣中西區'}
{'2353': '旅捷商務旅館 台灣苓雅'}
{'472': 'Z-house市政府捷運站套房 台灣信義區'}
{'1210': '雅思德商務公寓 台灣中和'}
{'4972': '奇豆民宿 台灣花蓮市'}


In [71]:
a = [list(i.keys())[0] for i in rcmhotel['rcmhotel']]
print(a)
a = [i.keys() for i in rcmhotel['rcmhotel']]
print(list(a))

['1984', '5914', '5233', '3394', '5042', '1800', '2353', '472', '1210', '4972']
[dict_keys(['1984']), dict_keys(['5914']), dict_keys(['5233']), dict_keys(['3394']), dict_keys(['5042']), dict_keys(['1800']), dict_keys(['2353']), dict_keys(['472']), dict_keys(['1210']), dict_keys(['4972'])]


In [68]:
for i in rcmhotel['rcmhotel']:
    print(i)

{'1984': '異宿風巢景觀旅店 台灣安平'}
{'5914': '長成民宿 台灣澎湖'}
{'5233': '花蓮莊家堡田園民宿 台灣吉安'}
{'3394': '綠原度假旅社 台灣雲林'}
{'5042': '逸心園民宿 台灣壽豐'}
{'1800': '旅人船說背包聚落 台灣中西區'}
{'2353': '旅捷商務旅館 台灣苓雅'}
{'472': 'Z-house市政府捷運站套房 台灣信義區'}
{'1210': '雅思德商務公寓 台灣中和'}
{'4972': '奇豆民宿 台灣花蓮市'}


In [84]:
rcmhotel_collection=mongo.db.user
rcmhotel = rcmhotel_collection.find_one({'account':'charooooonc'})
    
hotel_collection=mongo.db.hotel_id
hotelid = [list(i.keys())[0] for i in rcmhotel['rcmhotel']]#list
hotels = []
for i in hotelid:
    print(i)
    hotels.append(hotel_collection.find_one({'hotelid': i }))
print(hotels)

1984
5914
5233
3394
5042
1800
2353
472
1210
4972
[{'_id': ObjectId('5da2b87ad686edf8ccf3ceaa'), 'hotelid': '1984', 'location': '台灣安平', 'hotelname': '異宿風巢景觀旅店', 'orginalid': '14208219', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g13792460-d14208219-Reviews-Exotic_Homestay-Anping_Tainan.html', 'imgpath': 'https://media-cdn.tripadvisor.com/media/photo-s/15/26/ff/e8/caption.jpg', 'hotel_rating': '5.0'}, {'_id': ObjectId('5da2b87cd686edf8ccf3de04'), 'hotelid': '5914', 'location': '台灣澎湖', 'hotelname': '長成民宿', 'orginalid': '12813267', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g12420759-d12813267-Reviews-Chang_Cheng_Min_Su-Magong_Penghu.html', 'hotel_rating': '5.0'}, {'_id': ObjectId('5da2b87bd686edf8ccf3db5b'), 'hotelid': '5233', 'location': '台灣吉安', 'hotelname': '花蓮莊家堡田園民宿', 'orginalid': '2246569', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g13806642-d2246569-Reviews-Hualien_Chuangjiabao-Ji_an_Hualien.html', 'imgpath': 'https://media-cdn.tripadvisor.com/media/photo

In [94]:
hotel_collection=mongo.db.hotel_id
context = {#find()return list item
    'hotels':[i for i in hotel_collection.find({'_id':{'$gte':ObjectId('5da2b879d686edf8ccf3c6eb')}}).limit(10)]\
}

In [ ]:
5da2b878d686edf8ccf3c6eb
5da2b879d686edf8ccf3c6eb

In [23]:
price = '2'
print(str(int(price)*2))

4


In [19]:

rcmhotel_collection=mongo.db.user
rcmhotel = rcmhotel_collection.find_one({'account':'charooooonc'})
    
hotel_collection=mongo.db.hotel_id
hotelid = [list(i.keys())[0] for i in rcmhotel['rcmhotel']]#list
hotels = []
print(hotelid)
for i in hotelid:
    hotels.append(hotel_collection.find_one({'hotelid': int(i) }))
context = {
        'hotels':[i for i in hotels]\
    }
print(context)

['1984', '5914', '5233', '3394', '5042', '1800', '2353', '472', '1210', '4972']
{'hotels': [{'_id': ObjectId('5da2b87ad686edf8ccf3ceaa'), 'hotelid': 1984, 'location': '台灣安平', 'hotelname': '異宿風巢景觀旅店', 'orginalid': '14208219', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g13792460-d14208219-Reviews-Exotic_Homestay-Anping_Tainan.html', 'imgpath': 'https://media-cdn.tripadvisor.com/media/photo-s/15/26/ff/e8/caption.jpg', 'hotel_rating': '5.0'}, {'_id': ObjectId('5da2b87cd686edf8ccf3de04'), 'hotelid': 5914, 'location': '台灣澎湖', 'hotelname': '長成民宿', 'orginalid': '12813267', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g12420759-d12813267-Reviews-Chang_Cheng_Min_Su-Magong_Penghu.html', 'hotel_rating': '5.0'}, {'_id': ObjectId('5da2b87bd686edf8ccf3db5b'), 'hotelid': 5233, 'location': '台灣吉安', 'hotelname': '花蓮莊家堡田園民宿', 'orginalid': '2246569', 'url': 'https://www.tripadvisor.com.tw/Hotel_Review-g13806642-d2246569-Reviews-Hualien_Chuangjiabao-Ji_an_Hualien.html', 'imgpath': 'https://m

In [98]:
from web3 import Web3, HTTPProvider
w3 = Web3(HTTPProvider('http://localhost:7545'))


renthotel_collection=mongo.db.rent_hotel
user_collection=mongo.db.user
transaction_collection=mongo.db.transaction
publickey = user_collection.find_one({'account':'user2'}).get('public_key')
print(publickey)
a = [i for i in transaction_collection.find({'from':publickey.lower()}).sort('_id',-1)]#lower轉小寫字母

booked = []
print(len(a))
for i in a:
    print(i.get('hotelname'))
#     print(i.get('to'))#contractAddress
#     booked.append(renthotel_collection.find_one({'contractAddress':i.get('to')}))
    booked.append(renthotel_collection.find_one({'contractAddress':i.get('to')}))
    
# print(booked)



0x740AE084FE6A8B10923F36A1009124ae673caC37
4
ee
tt
yy
yu
